In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensornetwork as tn
import numpy as np
from tensorflow.keras.datasets import mnist
import time

tn.set_default_backend("numpy")

In [2]:
# The following code construct a MPS like this

#                        label_size
#                            |
    # bond_dim               |
# A1 --------- A2 ---------- L --------- A3 --------- An
# |            |                         |            |
# |            |                         |            |
#           feature      
def construct_MPS(bond_dim, feature, rank, label_size):
    label_site = rank // 2
    mps = tn.InfiniteMPS.random(
        d = [feature for _ in range(label_site)] + [label_size] + [feature for _ in range(rank - label_site)],
        D = [bond_dim for _ in range(rank+2)],
        dtype = np.float32
    )

    # left_region = np.stack(mps.tensors[:label_site], axis=0)
    # output_site = mps.tensors[label_site]
    # right_region = np.stack(mps.tensors[label_site+1:], axis=0)
    
    return mps
    # return nodes, connected_bonds

In [15]:
tf.keras.backend.set_floatx('float32')
class MPSModle(tf.keras.Model):
    def __init__(self, bond_dim, feature, rank, label_size):
        super(MPSModle, self).__init__()
        mps = construct_MPS(bond_dim, feature, rank, label_size)
        label_site = rank // 2
        left_input_tensors = mps.tensors[:label_site]
        output_tensor = mps.tensors[label_site]
        right_input_tensors = mps.tensors[label_site+1:]

        left_input_region = InputRegion(left_input_tensors)
        outputM = OutputModule(output_tensor)
        right_input_region = InputRegion(right_input_tensors)

        self.left_input_region = left_input_region
        self.outputM = outputM
        self.right_input_region = right_input_region
        self.label_site = label_site

    def __call__(self, x):
        # Break forward function so that we can parallelize a batch
        # using tf.vectorized_map
        # x [batch, rank, feature]
        def forward(input):
            # input [rank, feature]
            # left_rel [l,r]
            # output_rel [l, label_size, r]
            # right_rel [l,r]
            left_rel = self.left_input_region(input[:self.label_site])
            output_rel = self.outputM()
            right_rel = self.right_input_region(input[self.label_site:])
            left_rel.edges[0] ^ right_rel.edges[1]
            left_rel.edges[1] ^ output_rel.edges[0]
            output_rel.edges[2] ^ right_rel.edges[0]
            c = left_rel @ output_rel
            return (c @ right_rel).tensor

        # return forward(x)
        # return tf.vectorized_map(forward, x)
        return tf.map_fn(fn=lambda t: forward(t.numpy()), elems=tf.constant(x))


class InputModule(tf.keras.Model):
    def __init__(self, tensor, name=None):
        super().__init__(name=name)
        # tensor [l, feature, r]
        self.input_tensor = tf.Variable(tensor, trainable=True)

    def __call__(self, input):
        # input [feature]
        core = tn.Node(self.input_tensor.numpy())
        input_node = tn.Node(input)
        core.edges[1] ^ input_node[0]
        result = core @ input_node
        return result

class OutputModule(tf.keras.Model):
    def __init__(self, tensor, name=None):
        super().__init__(name=name)
        # tensor [l, label_size, r]
        self.output_tensor = tf.Variable(tensor, trainable=True)

    def __call__(self):        
        return tn.Node(self.output_tensor.numpy())

class InputRegion(tf.keras.Model):
    def __init__(self, tensors, name=None):
        super().__init__(name=name)
        # tensors [[l, feature, r]]
        input_Modules = [InputModule(tensor) for tensor in tensors]

        self.tensors = tensors
        self.input_Modules = input_Modules
    
    def __call__(self, input):
        # input [rank, feature]
        assert input.shape[0] == len(self.input_Modules)
        # nodes [[l,r]]
        nodes = [module(tensor) for module, tensor in zip(self.input_Modules, input)]
        # rel [l,r]
        for i in range(len(nodes) - 1):
            nodes[i].edges[1] ^ nodes[i+1].edges[0]
        return tn.contractors.greedy(nodes, output_edge_order=[nodes[0].edges[0], nodes[-1].edges[1]])

In [17]:
print("######################################")
print("FOR inputModule Test")
input_tensor = np.random.random_sample([3,2,3])
# input_tensor = tf.random.normal(shape=(3,2,3))
inputM = InputModule(input_tensor)
print(inputM.variables)
# print(inputM(tf.ones([2])).tensor.shape)
print(inputM(np.ones(2)).tensor.shape)

print("\n######################################")
print("FOR outputModule Test")
output_tensor = np.random.random_sample([3,10,3])
# output_tensor = tf.random.normal(shape=(3,10,3))
outputM = OutputModule(output_tensor)
print(outputM().tensor.shape[1])

print("\n######################################")
print("FOR inputRegion Test")
tensors = [np.random.random_sample([3,2,3]) for _ in range(10)]
# tensors = [tf.random.normal(shape=(3,2,3)) for _ in range(10)]
inputRegion = InputRegion(tensors)
print(len(inputRegion.variables))

input = np.random.random_sample([10,2])
# input = tf.random.normal(shape=(10,2))
print(inputRegion(input).tensor.shape)

print("\n######################################")
print("FOR MPS Test")
bond_dim = 3
feature = 2
rank = 28**2
label_size = 10
mps_Model = MPSModle(bond_dim, feature, rank, label_size)
print(len(mps_Model.variables))

images = np.random.random_sample([128,28**2,2])
print(mps_Model(images).shape)

######################################
FOR inputModule Test
[<tf.Variable 'Variable:0' shape=(3, 2, 3) dtype=float64, numpy=
array([[[0.47603843, 0.90917481, 0.91551441],
        [0.6745569 , 0.03120589, 0.4739484 ]],

       [[0.85518248, 0.852788  , 0.06698272],
        [0.87361515, 0.87990734, 0.98914353]],

       [[0.74696419, 0.45917349, 0.57238012],
        [0.51739295, 0.46322423, 0.11109172]]])>]
(3, 3)

######################################
FOR outputModule Test
10

######################################
FOR inputRegion Test
10
(3, 3)

######################################
FOR MPS Test
785
(128, 10)
